In [3]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path, shift_skeleton
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab
from sparse_util import dilate, zhangSuen
from realign import realign, reconnect
from util import get_path
import pandas as pd

In [4]:
# date0='0709_1157'
# date1='0709_1557'
# date2='0709_1934'
date0='0701_1957'
date1='0701_2357'
date2='0702_0357'
date3='0702_0757'
date4='0702_1157'
date5='0702_1557'
plate=13
dates=[date0,date1,date2,date3,date4,date5]


In [ ]:
mat_skels=[read_mat(get_path(date,plate,True))['skel'] for date in dates]

In [5]:
dic_skels=[dic_to_sparse(mat_skel) for mat_skel in mat_skels]

In [6]:
skeleton_docs=[sparse.dok_matrix(dic_skel) for dic_skel in dic_skels]

In [ ]:
sub_skeleton_docs_aligned = [realign(skeleton_doc,skeleton_docs[0]) for skeleton_doc in skeleton_docs]

In [8]:
skeleton_docs_aligned = [reconnect(skeleton_docs[0])]+sub_skeleton_docs_aligned 

872558 872271
107117 796
394 60
35 28
17 15
10 0
0 0


In [10]:
for i, date in enumerate([date0,date1,date2,date3,date4,date5]):
    sparse.save_npz(f'Data/skeleton_{date}_{plate}_full',sparse.csc_matrix(skeleton_docs_aligned[i]))

In [6]:
skeleton_docs_aligned = [sparse.dok_matrix(sparse.load_npz(f'Data/skeleton_{date}_{plate}_full.npz')) for date in dates]

In [7]:
graph_tabs=[from_sparse_to_graph(skeleton) for skeleton in skeleton_docs_aligned]

In [8]:
nx_graph_poss=[generate_nx_graph(graph_tab) for graph_tab in graph_tabs]
nx_graphs_aligned=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
nx_graph_pruned=[prune_graph(nx_graph) for nx_graph in nx_graphs_aligned]

In [9]:
for i in range (len(dates)-1,0,-1):
    poss_aligned[i+1], nx_graph_pruned[i+1], nx_graph_pruned[i], corresp = second_identification(nx_graph_pruned[i],nx_graph_pruned[i+1],poss_aligned[i],poss_aligned[i+1],50)

IndexError: list index out of range

In [32]:
skeleton_docs_cleaned= [generate_skeleton(nx_graph,skeleton_docs_aligned[i].shape) for i,nx_graph in enumerate(nx_graph_pruned)]

In [33]:
for i, date in enumerate([date0,date1,date2,date3,date4,date5]):
    sparse.save_npz(f'Data/skeleton_{date}_{plate}_full_clean',sparse.csc_matrix(skeleton_docs_cleaned[i]))

In [26]:
def from_nx_to_tab(nx_graph,pos):
    column_names = ["origin_label","end_label","origin_pos", "end_pos", "pixel_list"]
    tab = pd.DataFrame(columns=column_names)
    for edge in nx_graph.edges:
        origin_label=edge[0]
        end_label=edge[1]
        origin_pos=pos[origin_label]
        end_pos = pos[end_label]
        pixel_list=orient(nx_graph.get_edge_data(*edge)['pixel_list'],origin_pos)
        new_line=pd.DataFrame({"origin_label":[origin_label],"end_label" : [end_label], "origin_pos":[origin_pos], "end_pos" : [end_pos], "pixel_list" : [pixel_list]})
        tab=tab.append(new_line,ignore_index=True)
    return(tab)

In [34]:
tabs_labeled = [from_nx_to_tab(nx_graph,poss_aligned[i]) for i, nx_graph in enumerate(nx_graph_pruned)]

In [35]:
tabs = [from_sparse_to_graph(skeleton) for skeleton in skeleton_docs_cleaned]

In [36]:
for i, date in enumerate(dates):
    tabs_labeled[i].to_csv(get_path(date,plate,True,extension='_full_labeled.csv'))
    tabs[i].to_csv(get_path(date,plate,True,extension='_full.csv'))
    

In [ ]:
for i, date in enumerate(dates):
    

In [ ]:
tab_labeled_t0m2.to_csv(get_path(date0,plate,True,extension='_full_labeled.csv'))
tab_labeled_t0m1.to_csv(get_path(date1,plate,True,extension='_full_labeled.csv'))
tab_labeled_t0.to_csv(get_path(date2,plate,True,extension='_full_labeled.csv'))

In [ ]:
graph_t0m2.to_csv(get_path(date0,plate,True,extension='_full.csv'))
graph_t0m1.to_csv(get_path(date1,plate,True,extension='_full.csv'))
graph_t0.to_csv(get_path(date2,plate,True,extension='_full.csv'))

In [ ]:
tab_labeled_t0m2.to_csv(f'Data/graph_{date0}_{plate}_full_labeled.csv')
tab_labeled_t0m1.to_csv(f'Data/graph_{date1}_{plate}_full_labeled.csv')
tab_labeled_t0.to_csv(f'Data/graph_{date2}_{plate}_full_labeled.csv')

In [ ]:
graph_t0m2.to_csv(f'Data/graph_{date0}_{plate}_full.csv')
graph_t0m1.to_csv(f'Data/graph_{date1}_{plate}_full.csv')
graph_t0.to_csv(f'Data/graph_{date2}_{plate}_full.csv')

In [3]:
tab_labeled_t0m2=pd.read_csv(f'Data/graph_{date0}_{plate}_full_labeled.csv')
tab_labeled_t0m1=pd.read_csv(f'Data/graph_{date1}_{plate}_full_labeled.csv')
tab_labeled_t0=pd.read_csv(f'Data/graph_{date2}_{plate}_full_labeled.csv')

In [5]:
graph_t0m2=pd.read_csv(f'Data/graph_{date0}_{plate}_full.csv')
graph_t0m1=pd.read_csv(f'Data/graph_{date1}_{plate}_full.csv')
graph_t0=pd.read_csv(f'Data/graph_{date2}_{plate}_full.csv')

In [ ]:
tab_labeled_t0m2.loc[tab_labeled_t0m2['origin_label']==300]

In [ ]:
tab_labeled_t0m1.loc[tab_labeled_t0m1['origin_label']==300]

In [180]:
sub_skeleton_t0m1=skeleton_t0m1[begin:end,begin:end]


In [181]:
sub_skeleton_t0=skeleton_t0[begin:end,begin:end]

In [182]:
list_node_t0m1=[node for node in anx_graph_t0m1 if (begin<pos_1[node][0]<end and begin<pos_1[node][1]<end)]

In [183]:
list_node_t0=[node for node in anx_graph_t0 if (begin<apos_t0[node][0]<end and begin<apos_t0[node][1]<end)]

In [185]:
plot_t_tp1(list_node_t0m1,list_node_t0,pos_1,apos_t0,sub_skeleton_t0m1.todense(),sub_skeleton_t0.todense(),shift=(begin,begin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [147]:
plot_t_tp1([],[],pos_1,apos_t0,skeleton1.todense(),skeleton2.todense())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
np.ceil(4.3)

5.0

In [41]:
from scipy.optimize import minimize

In [107]:
sub_skeleton_t0m1.shape

(5000, 5000)

In [172]:
shift_sub_skeleton_t0m1=shift_skeleton(sub_skeleton_t0m1,shift)

In [104]:
np.round(5.4)

5.0

In [ ]:
def shift_skeleton(skeleton,shift):
    shifted_skeleton=sparse.dok_matrix(skeleton.shape, dtype=bool)
    for pixel in skeleton.keys():
#             print(pixel[0]+shift[0],pixel[1]+shift[1])
            if (skeleton.shape[0]>np.ceil(pixel[0]+shift[0])>0 and skeleton.shape[1]>np.ceil(pixel[1]+shift[1])>0):
                shifted_pixel = (np.round(pixel[0]+shift[0]),np.round(pixel[1]+shift[1]))
                shifted_skeleton[shifted_pixel]=1
    return(shifted_skeleton)

In [40]:
skeleton_t0-skeleton_t0m1

<20687x45556 sparse matrix of type '<class 'numpy.bool_'>'
	with 2683488 stored elements in Compressed Sparse Row format>